In [1]:
import os

import numpy as np
import pysindy as ps

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import utils

In [2]:
dataset_folder = "D:/Robotics/neural-fly-main/neural-fly-main/data/experiment"

features1 = ['v']
RawData1 = utils.load_data(dataset_folder)
Data1 = utils.format_data(RawData1, features=features1)

In [118]:
v = torch.zeros((2511,3)) #first input feature
for row in range(len(v)):
    for column in range(3):
        v[row][column] = Data1[21][0][row][column]
        
v.requires_grad = True

In [119]:
features2 = ['T_sp']

Data2 = utils.format_data(RawData1, features=features2)

fu = torch.zeros((2511,3))
for row in range(len(fu)):
    fu[row][2] = Data2[21][0][row][0]
    
R = torch.zeros((2511,3,3))
features3 = ['R']

Data3 = utils.format_data(RawData1, features=features3)

for matrix in range(len(R)):
    for row in range(3):
        for column in range(3):
            R[matrix][row][column] = Data3[21][0][matrix][row][column]
            
Rfu = torch.zeros((2511,3)) #second input feature
for row in range(len(Rfu)):
    Rfu[row] = np.matmul(R[row],fu[row])
Rfu.shape
Rfu.requires_grad = True

In [120]:
acc = torch.zeros((2511,3)) # actual acceleration
for ele in range(2510):
    acc[ele] = (v[ele+1]-v[ele])/0.02
acc[2510] = acc[2509]

In [121]:
class Regression(nn.Module):
    def __init__(self, input_size, output_size):
        super(Regression, self).__init__()
        self.linear1 = nn.Linear(input_size, 8)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(8, 16)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(16, 24)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(24, 8)
        self.relu4 = nn.ReLU()
        self.linear5 = nn.Linear(8, output_size)
    
    def forward(self, x):
        out = x
        out = out.unsqueeze(1)
        out = self.linear1(out)
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        out = self.linear3(out)
        out = self.relu3(out)
        out = self.linear4(out)
        out = self.relu4(out)
        out = self.linear5(out)
        return out
    
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)
    
model = Regression(1,2)

model.apply(init_weights)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [122]:
f1 = model(v[:,0])[:,0]
f2 = model(v[:,1])[:,0]
f3 = model(v[:,2])[:,0]
f4 = model(v[:,0])[:,1]
f5 = model(v[:,1])[:,1]
f6 = model(v[:,2])[:,1]
f7 = model(Rfu[:,0])[:,0]
f8 = model(Rfu[:,1])[:,0]
f9 = model(Rfu[:,2])[:,0]
f10 = model(Rfu[:,0])[:,1]
f11 = model(Rfu[:,1])[:,1]
f12 = model(Rfu[:,2])[:,1]
feature = torch.cat((f1.unsqueeze(1), f2.unsqueeze(1), f3.unsqueeze(1), f4.unsqueeze(1), f5.unsqueeze(1), f6.unsqueeze(1), f7.unsqueeze(1), f8.unsqueeze(1), f9.unsqueeze(1), f10.unsqueeze(1), f11.unsqueeze(1), f12.unsqueeze(1)), dim=1)
feature.shape

torch.Size([2511, 12])

In [123]:
E = torch.randn(12, 3)
E.requires_grad = True
E.shape

torch.Size([12, 3])

In [124]:
y = torch.matmul(feature, E)
y.requires_grad

True

In [125]:
loss = criterion(acc,y)
loss

tensor(1.5547, grad_fn=<MseLossBackward0>)

In [126]:
E

tensor([[-1.0266,  1.2158, -1.0716],
        [ 0.2786, -0.6571, -0.1378],
        [ 0.6623, -0.6114,  0.2539],
        [-1.1110, -1.4723, -1.3932],
        [ 1.3906, -0.2877, -1.9481],
        [ 0.6072,  0.3241,  0.3054],
        [ 0.7300, -0.6628, -0.8665],
        [-0.1969, -1.6452, -1.7888],
        [-0.7478,  0.2617, -0.5976],
        [ 0.4532,  0.8352,  0.0423],
        [-0.1394,  1.2539, -0.3139],
        [ 0.1562,  0.9522,  0.7631]], requires_grad=True)

In [127]:
loss.backward()

In [146]:
E.grad

tensor([[-4.0883e-03,  6.7414e-03,  1.9197e-03],
        [-1.9396e-04,  7.8724e-05, -3.3767e-04],
        [-1.8543e-03,  3.8087e-03, -1.1123e-02],
        [ 1.2564e-03, -2.0723e-03, -5.9737e-04],
        [ 5.8873e-05, -2.4649e-05,  1.0204e-04],
        [ 5.7716e-04, -1.1862e-03,  3.4056e-03],
        [-3.0918e-03,  3.4369e-04, -2.1660e-03],
        [-9.2902e-05,  4.1947e-05, -3.4310e-04],
        [-3.3725e-03,  2.7748e-03, -2.1733e-02],
        [ 9.5647e-04, -1.0631e-04,  6.7009e-04],
        [ 2.8735e-05, -1.2974e-05,  1.0612e-04],
        [ 1.0431e-03, -8.5828e-04,  6.7220e-03]])

In [147]:
optimizer2 = torch.optim.SGD([E], lr=0.1)
optimizer2.step()
optimizer2.zero_grad()

In [150]:
print(E.grad)

None


In [129]:
optimizer.step()

In [130]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.grad)


linear1.weight tensor([[ 0.0476],
        [ 0.0037],
        [ 0.0440],
        [-0.0207],
        [ 0.0465],
        [-0.0050],
        [-0.0034],
        [-0.0042]])
linear1.bias tensor([ 0.0308,  0.0017,  0.0360,  0.0161,  0.0167,  0.0422, -0.0167,  0.0232])
linear2.weight tensor([[-5.1690e-03,  1.6522e-05, -1.2532e-02, -2.1687e-02, -6.5349e-03,
          1.1110e-04,  9.9960e-04,  6.1736e-04],
        [-2.1035e-03, -1.5953e-05, -5.0999e-03, -8.8253e-03, -2.6593e-03,
         -1.0727e-04, -9.6517e-04, -5.9609e-04],
        [ 0.0000e+00,  1.8033e-05,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.2126e-04,  1.0910e-03,  6.7382e-04],
        [-1.5792e-02, -2.0518e-05, -3.8287e-02, -6.6255e-02, -1.9964e-02,
         -1.3796e-04, -1.2413e-03, -7.6664e-04],
        [ 7.8815e-03,  7.1965e-05,  1.9109e-02,  3.3067e-02,  9.9642e-03,
          4.8389e-04,  4.3539e-03,  2.6890e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,